In [1]:
import numpy as np
from functions import greedy_cycle_with_regret, local_search_steepest

In [2]:
def mutate(parent1: np.array, parent2: np.array, **kwargs):
    common_nodes = np.intersect1d(parent1,parent2)
    all_nodes = np.arange(200)
    np.random.shuffle(all_nodes)
    common_nodes_mask = np.apply_along_axis(lambda x: np.isin(x,common_nodes),axis=0,arr=parent1)
    parent1[~common_nodes_mask] = np.setdiff1d(all_nodes,common_nodes)[:np.count_nonzero(~common_nodes_mask)]
    return parent1

In [3]:
def mutate2(parent1: np.array,
            parent2: np.array,
            distances: np.ndarray,
            data:np.ndarray,
            run_local_search: bool = True):
    common_nodes = np.intersect1d(parent1,parent2)
    common_nodes_mask = np.apply_along_axis(lambda x: np.isin(x,common_nodes),axis=0,arr=parent1)
    offspring =  greedy_cycle_with_regret(parent1[common_nodes_mask], distances, data, 3, 0.5)
    if run_local_search:
        offspring = local_search_steepest(offspring, distances, data)
    return offspring

In [ ]:
offspring = mutate(parent1=parent1,
                   parent2=parent2,
                   distances=distances,
                   data=data,
                   run_local_search=False)